# 2020.03.28 (Sat)

## Base model: spaCy

In [1]:
!prodigy train ner cord_19_abstracts_match models/en_core_web_lg_no_ner --output models/2020_03_28_match/en_rf_web_lg

✔ Loaded model 'models/en_core_web_lg_no_ner'
Created and merged data for 86 total examples
Using 43 train / 43 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    2260.23       3.822      5.714      4.580                                 
 2     304.20      96.203     72.381     82.609                                 
 3      35.94      93.069     89.524     91.262                                 
 4       7.71      91.262     89.524     90.385                                 
 5       6.89      94.000     89.524     91.707                                 
 6       6.26      94.949     89.524     92.157                                 
 7       4.15      94.000     89.524     91.707                                 
 8      10.58      92.157   

## Base model: scispaCy

In [2]:
! prodigy train ner cord_19_abstracts_match models/en_core_sci_lg_no_ner --output models/2020_03_28_match/en_rf_sci_lg

✔ Loaded model 'models/en_core_sci_lg_no_ner'
Created and merged data for 86 total examples
Using 43 train / 43 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    1864.33       3.333      2.941      3.125                                 
 2     303.79      93.902     75.490     83.696                                 
 3      22.39      92.784     88.235     90.452                                 
 4      17.47      94.792     89.216     91.919                                 
 5       5.36      93.814     89.216     91.457                                 
 6       6.78      92.857     89.216     91.000                                 
 7       6.09      93.814     89.216     91.457                                 
 8       4.87      94.792   